# Implementación y Despliegue del modelo predictivo

Se cargan las librerías necesarias

In [1]:
import pyodbc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sodapy import Socrata


import warnings
warnings.filterwarnings('ignore')

from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from joblib import dump, load

## Cargando los datos

In [2]:
def iniciarSesion():
    server = 'tcp:Miadgpproyectofinal.database.windows.net' 
    database = 'DB_GP_PROYECTO' 
    username = 'admin_miad' 
    password = 'Abc123456789' 
    
    cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
    cursor = cnxn.cursor()
    
    return cnxn, cursor

In [3]:
def cerrarCursor(cursor):
    try:
        cursor.close()
    except:
        pass

In [9]:
cnxn, cursor = iniciarSesion()

In [10]:
sql = """
SELECT *
FROM Prod_SabanaAnalitica
"""

In [5]:
df = pd.read_sql(sql,cnxn)
df

,Pro_Entidad,Pro_Nit Entidad,Pro_Departamento Entidad,Pro_Ciudad Entidad,Pro_OrdenEntidad,Pro_Entidad Centralizada,Pro_ID del Proceso,Pro_Referencia del Proceso,Pro_PCI,Pro_ID del Portafolio,...,Con_Valor Pendiente de Pago,Con_Valor Pagado,Con_Valor Amortizado,Con_Valor Pendiente de Amortizacion,Con_Valor Pendiente de Ejecucion,Con_Dias Adicionados,Con_Fecha Inicio Liquidacion,Con_Fecha Fin Liquidacion,Cat_Extraccion,FueMultado
0,811021743,811021743,Antioquia,Medellín,Territorial,No,CO1.REQ.3314776,1127,ND,CO1.BDOS.3228167,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cruce Directo ProcesoContrato,0
1,811021743,811021743,Antioquia,Medellín,Territorial,No,CO1.REQ.3613724,1129,ND,CO1.BDOS.3520939,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cruce Directo ProcesoContrato,0
2,811021743,811021743,Antioquia,Medellín,Territorial,No,CO1.REQ.4355856,1139,ND,CO1.BDOS.4256735,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cruce Directo ProcesoContrato,0
3,ACUEDUCTO METROPOLITANO DE BUCARAMANGA SA ESP,8902001622,No Definido,No Definido,Territorial,Si,CO1.REQ.3648589,CO249-2022,ND,CO1.BDOS.3555063,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cruce Directo ProcesoContrato,0
4,ACUEDUCTO METROPOLITANO DE BUCARAMANGA SA ESP,8902001622,No Definido,No Definido,Territorial,Si,CO1.REQ.3775787,CO 293-22,ND,CO1.BDOS.3679782,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cruce Directo ProcesoContrato,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46957,Universidad del Atlántico,890102257,Atlántico,Puerto Colombia,Nacional,Descentralizada,No Aplica,No Aplica,No Aplica,No Aplica,...,89804819.0,0.0,0.0,44902409.0,89804819.0,0.0,2023-04-30,2023-06-01,Inconsistencias de Proveedores,0
46958,Universidad del Atlántico,890102257,Atlántico,Puerto Colombia,Nacional,Descentralizada,No Aplica,No Aplica,No Aplica,No Aplica,...,227762890.0,0.0,0.0,113881445.0,227762890.0,0.0,2023-04-30,2023-05-31,Inconsistencias de Proveedores,0
46959,Universidad del Atlántico,890102257,Atlántico,Puerto Colombia,Nacional,Descentralizada,No Aplica,No Aplica,No Aplica,No Aplica,...,99890065.0,0.0,0.0,49945032.0,99890065.0,0.0,2022-04-30,2022-05-31,Inconsistencias de Proveedores,0
46960,Universidad del Atlántico,890102257,Atlántico,Puerto Colombia,Nacional,Descentralizada,No Aplica,No Aplica,No Aplica,No Aplica,...,299447468.0,0.0,0.0,0.0,299447468.0,0.0,2023-05-01,2023-06-01,Inconsistencias de Proveedores,0


In [81]:
df = pd.read_csv('sabana_completa.csv')

Arreglamos la variable de salida como 0 cuando viene de tipo Null

In [82]:
df['FueMultado'] = df['FueMultado'].replace(np.nan, 0)
df['FueMultado'] = df['FueMultado'].replace({'0': 0, 'Multado': 1})
df['FueMultado'] = df['FueMultado'].astype('int64')

Partimos los datos en set de entrenamiento como todos los proyectos anteriores a 2023 y como set de validación los proyectos de 2023 en adelante

In [83]:
df_train = df
df_train['Pro_Fecha de Publicacion del Proceso']= pd.to_datetime(df_train['Pro_Fecha de Publicacion del Proceso'])
df_train['Year'] = df_train['Pro_Fecha de Publicacion del Proceso'].dt.strftime('%Y')
df_train['Year'] = df_train['Year'].fillna(0)
df_train['Year'] = df_train['Year'].astype('int')
df_test = df_train[df_train['Year'] >= 2023]
df_train = df_train[df_train['Year'] < 2023]

Revisamos que la base se encuentre en igualdad de dimensiones contra la que se tiene en la BD

In [84]:
num_filas = df.shape[0]
num_columnas = df.shape[1]
print("El DataFrame tiene", num_filas, "filas y", num_columnas, "columnas.")
num_filas = df_train.shape[0]
num_columnas = df_train.shape[1]
print("El DataFrame de entrenamiento tiene", num_filas, "filas y", num_columnas, "columnas.")
num_filas = df_test.shape[0]
num_columnas = df_test.shape[1]
print("El DataFrame de validación tiene", num_filas, "filas y", num_columnas, "columnas.")

El DataFrame tiene 46962 filas y 81 columnas.
El DataFrame de entrenamiento tiene 43714 filas y 81 columnas.
El DataFrame de validación tiene 3248 filas y 81 columnas.


Seleccionamos las columnas a utilizar en el ejercicio

In [85]:
cols_filt = ['Pro_Entidad', 'Pro_Nit Entidad', 'Pro_Departamento Entidad',
       'Pro_Ciudad Entidad', 'Pro_OrdenEntidad', 'Pro_Entidad Centralizada',
       'Pro_ID del Proceso', 'Pro_Referencia del Proceso', 'Pro_PCI',
       'Pro_ID del Portafolio', 'Pro_Nombre del Procedimiento', 'Pro_Fase',
       'Pro_Fecha de Publicacion del Proceso',
       'Pro_Fecha de Ultima Publicación', 'Pro_Precio Base',
       'Pro_Modalidad de Contratacion',
       'Pro_Justificación Modalidad de Contratación', 'Pro_Duracion',
       'Pro_Unidad de Duracion', 'Pro_Ciudad de la Unidad de Contratación',
       'Pro_Nombre de la Unidad de Contratación', 'Pro_Proveedores Invitados',
       'Pro_Proveedores con Invitacion Directa',
       'Pro_Visualizaciones del Procedimiento',
       'Pro_Proveedores que Manifestaron Interes',
       'Pro_Respuestas al Procedimiento', 'Pro_Respuestas Externas',
       'Pro_Conteo de Respuestas a Ofertas',
       'Pro_Proveedores Unicos con Respuestas', 'Pro_Numero de Lotes',
       'Pro_Estado del Procedimiento', 'Pro_ID Estado del Procedimiento',
       'Pro_Adjudicado', 'Pro_ID Adjudicacion', 'Pro_CodigoProveedor',
       'Pro_Departamento Proveedor', 'Pro_Ciudad Proveedor',
       'Pro_Valor Total Adjudicacion', 'Pro_Nombre del Adjudicador',
       'Pro_Nombre del Proveedor Adjudicado',
       'Pro_NIT del Proveedor Adjudicado', 'Pro_Codigo Principal de Categoria',
       'Pro_Estado de Apertura del Proceso', 'Pro_Tipo de Contrato',
       'Pro_Subtipo de Contrato', 'Pro_URLProceso', 'Pro_Codigo Entidad',
       'Pro_Estado Resumen', 'Cat_Extraccion', 'FueMultado']

df_train = df_train[cols_filt]
df_test = df_test[cols_filt]

cat_cols = ['Pro_Departamento Entidad', 'Pro_OrdenEntidad', 'Pro_Entidad Centralizada', 'Pro_Fase', 'Pro_Modalidad de Contratacion', 'Pro_Justificación Modalidad de Contratación', 'Pro_Unidad de Duracion', 'Pro_Estado del Procedimiento', 'Pro_Adjudicado', 'Pro_Departamento Proveedor', 'Pro_Estado de Apertura del Proceso', 'Pro_Tipo de Contrato', 'Pro_Subtipo de Contrato', 'Pro_Estado Resumen', 'Cat_Extraccion', 'FueMultado']
num_cols = ['Pro_Nit Entidad', 'Pro_Precio Base', 'Pro_Duracion',
       'Pro_Proveedores Invitados', 'Pro_Proveedores con Invitacion Directa',
       'Pro_Visualizaciones del Procedimiento',
       'Pro_Proveedores que Manifestaron Interes',
       'Pro_Respuestas al Procedimiento', 'Pro_Respuestas Externas',
       'Pro_Conteo de Respuestas a Ofertas',
       'Pro_Proveedores Unicos con Respuestas', 'Pro_Numero de Lotes',
       'Pro_ID Estado del Procedimiento', 'Pro_Valor Total Adjudicacion',
       'Pro_Codigo Entidad']

df_train_final = pd.concat([df_train[num_cols], df_train[cat_cols]], axis=1)
df_test_final = pd.concat([df_test[num_cols], df_test[cat_cols]], axis=1)

Eliminamos unas columnas que no van a ser de utilidad para la predicción

In [86]:
df_train_final = df_train_final.drop('Pro_Modalidad de Contratacion', axis=1)
df_train_final = df_train_final.drop('Pro_Unidad de Duracion', axis=1)
df_test_final = df_test_final.drop('Pro_Modalidad de Contratacion', axis=1)
df_test_final = df_test_final.drop('Pro_Unidad de Duracion', axis=1)
# df_final = df_final.drop(['Pro_Fecha de Ultima Publicación', 'Pro_Fecha de Publicacion del Proceso'], axis=1)

Hacemos el proceso de sacar los dummies para las variables categoricas, haciendo uso del modelo OH ya almacenado anteriormente

In [87]:
# dump(enc, 'encod_oh.joblib') # save the model

In [116]:
# Seleccionar las columnas que son categóricas
cat_cols = ['Pro_Departamento Entidad', 'Pro_OrdenEntidad', 'Pro_Entidad Centralizada', 'Pro_Fase', 'Pro_Justificación Modalidad de Contratación', 'Pro_Estado del Procedimiento', 'Pro_Adjudicado', 'Pro_Departamento Proveedor', 'Pro_Estado de Apertura del Proceso', 'Pro_Tipo de Contrato', 'Pro_Subtipo de Contrato', 'Pro_Estado Resumen', 'Cat_Extraccion']

# one hot encoding
enc = load('encod_oh.joblib')
train_onehot = enc.transform(df_train_final[cat_cols])
test_onehot = enc.transform(df_test_final[cat_cols])

oh_df_train = pd.DataFrame(train_onehot)
oh_df_test = pd.DataFrame(test_onehot)

num_cols = ['Pro_Nit Entidad', 'Pro_Precio Base', 'Pro_Duracion',
       'Pro_Proveedores Invitados', 'Pro_Proveedores con Invitacion Directa',
       'Pro_Visualizaciones del Procedimiento',
       'Pro_Proveedores que Manifestaron Interes',
       'Pro_Respuestas al Procedimiento', 'Pro_Respuestas Externas',
       'Pro_Conteo de Respuestas a Ofertas',
       'Pro_Proveedores Unicos con Respuestas', 'Pro_Numero de Lotes',
       'Pro_ID Estado del Procedimiento', 'Pro_Valor Total Adjudicacion',
       'Pro_Codigo Entidad', 'FueMultado']

numeric_df_train = df_train_final[num_cols]
cols_names = list(numeric_df_train.columns) + list(oh_df_train.columns)
numeric_df_test = df_test_final[num_cols]

numeric_df_train.reset_index(drop=True, inplace=True)
oh_df_train.reset_index(drop=True, inplace=True)
new_df_train = pd.concat([numeric_df_train, oh_df_train], ignore_index=True, axis=1)
new_df_train.columns = cols_names
numeric_df_test.reset_index(drop=True, inplace=True)
oh_df_test.reset_index(drop=True, inplace=True)
new_df_test = pd.concat([numeric_df_test, oh_df_test], ignore_index=True, axis=1)
new_df_test.columns = cols_names

Hacemos imputación de datos para los valores nulos, debido a que estos no los puede interpretar el modelo final implementado en el desarrollo del proyecto.

In [90]:
# dump(imputer, 'imputer_ok.joblib') # save the model

In [119]:
# Construimos el modelo
# imputer = KNNImputer(n_neighbors=5, weights="uniform")
new_df_train.columns = new_df_train.columns.astype(str)
new_df_test.columns = new_df_test.columns.astype(str)

X_new_df_train = new_df_train[new_df_train.columns.drop(['FueMultado'])]
X_new_df_test = new_df_test[new_df_test.columns.drop(['FueMultado'])]

imputer = load('imputer_ok.joblib')
# new_df_2 = imputer.transform(new_df)
print('inicia')
X_new_train = imputer.transform(X_new_df_train)
print('termina')

inicia
termina


In [120]:
print('inicia')
X_new_test = imputer.transform(X_new_df_test)
print('termina')

inicia
termina


In [121]:
X_new_df_train = pd.DataFrame(X_new_train)
X_new_df_test = pd.DataFrame(X_new_test)

In [124]:
X_new_df_train['FueMultado'] = new_df_train['FueMultado']
X_new_df_test['FueMultado'] = new_df_test['FueMultado']

In [127]:
X_new_df_train.to_csv('sabana_completa_imputada_todo_train.csv')
X_new_df_test.to_csv('sabana_completa_imputada_todo_test.csv')

In [128]:
X_train = X_new_df_train[X_new_df_train.columns.drop(['FueMultado'])]
y_train = X_new_df_train['FueMultado']

X_test = X_new_df_test[X_new_df_test.columns.drop(['FueMultado'])]
y_test = X_new_df_test['FueMultado']

In [129]:
X_train.shape

(43714, 147)

In [130]:
X_test.shape

(3248, 147)

In [131]:
y_train.shape

(43714,)

In [33]:
# dump(scaler, 'scaler_ok.joblib') # save the model

['scaler_ok.joblib']

In [132]:
# scaler = StandardScaler()
scaler = load('scaler_ok.joblib')
X_scaled_train = scaler.transform(X_train)
X_scaled_train = pd.DataFrame(X_scaled_train)

X_scaled_test = scaler.transform(X_test)
X_scaled_test = pd.DataFrame(X_scaled_test)

In [43]:
# dump(pca, 'pca_ok.joblib') # save the model

['pca_ok.joblib']

In [133]:
# pca = PCA(n_components=0.95, random_state=42)
pca = load('pca_ok.joblib')
X_scaled_pca_train = pca.transform(X_scaled_train)
X_scaled_pca_train = pd.DataFrame(X_scaled_pca_train)

X_scaled_pca_test = pca.transform(X_scaled_test)
X_scaled_pca_test = pd.DataFrame(X_scaled_pca_test)

In [135]:
dump(rfc, 'rfc_2022_ok.joblib') # save the model

['rfc_2022_ok.joblib']

In [136]:
# rfc = RandomForestClassifier(n_estimators=100, min_samples_split=2, max_depth=5, random_state=42)
rfc = load('rfc_2022_ok.joblib')
# rfc.fit(X_scaled_pca_train, y_train)
probs = rfc.predict_proba(X_scaled_pca_test)
probs_1 = [i[1] for i in probs]

In [138]:
df_test['Prob predicción'] = probs_1

In [152]:
def definirColorSemaforo(num, bajo = 0.2, medio = 0.5):
    if num < bajo:
        return 'Sin Riesgo', 0
    elif num < medio:
        return 'Riesgo moderado', 1
    else:
        return 'Riesgo alto', 2

df_test['Descrip Semaforo'] = [definirColorSemaforo(i)[0] for i in df_test['Prob predicción']]
df_test['Semaforo'] = [definirColorSemaforo(i)[1] for i in df_test['Prob predicción']]

In [153]:
df_test.head(3)

,Pro_Entidad,Pro_Nit Entidad,Pro_Departamento Entidad,Pro_Ciudad Entidad,Pro_OrdenEntidad,Pro_Entidad Centralizada,Pro_ID del Proceso,Pro_Referencia del Proceso,Pro_PCI,Pro_ID del Portafolio,...,Pro_Tipo de Contrato,Pro_Subtipo de Contrato,Pro_URLProceso,Pro_Codigo Entidad,Pro_Estado Resumen,Cat_Extraccion,FueMultado,Prob predicción,Descrip Semaforo,Semaforo
2,811021743,811021743,Antioquia,Medellín,Territorial,No,CO1.REQ.4355856,1139,ND,CO1.BDOS.4256735,...,Obra,No Especificado,{'url': 'https://community.secop.gov.co/Public...,718522014,Presentación de oferta,Cruce Directo ProcesoContrato,0,0.000674,Sin Riesgo,0
11,ACUEDUCTO Y ALCANTARILLADO DE POPAYAN SA ESP,891500117,Cauca,Popayán,Territorial,Si,CO1.REQ.3583248,523 - 2022,ND,CO1.BDOS.3491164,...,Obra,No Especificado,{'url': 'https://community.secop.gov.co/Public...,704996446,Presentación de oferta,Cruce Directo ProcesoContrato,0,0.000847,Sin Riesgo,0
14,ACUEDUCTO Y ALCANTARILLADO DE POPAYAN SA ESP,891500117,Cauca,Popayán,Territorial,Si,CO1.REQ.3791184,546 - 2022,ND,CO1.BDOS.3695151,...,Obra,No Especificado,{'url': 'https://community.secop.gov.co/Public...,704996446,Presentación de oferta,Cruce Directo ProcesoContrato,0,0.000847,Sin Riesgo,0


In [141]:
list(df_test.columns)

['Pro_Entidad',
 'Pro_Nit Entidad',
 'Pro_Departamento Entidad',
 'Pro_Ciudad Entidad',
 'Pro_OrdenEntidad',
 'Pro_Entidad Centralizada',
 'Pro_ID del Proceso',
 'Pro_Referencia del Proceso',
 'Pro_PCI',
 'Pro_ID del Portafolio',
 'Pro_Nombre del Procedimiento',
 'Pro_Fase',
 'Pro_Fecha de Publicacion del Proceso',
 'Pro_Fecha de Ultima Publicación',
 'Pro_Precio Base',
 'Pro_Modalidad de Contratacion',
 'Pro_Justificación Modalidad de Contratación',
 'Pro_Duracion',
 'Pro_Unidad de Duracion',
 'Pro_Ciudad de la Unidad de Contratación',
 'Pro_Nombre de la Unidad de Contratación',
 'Pro_Proveedores Invitados',
 'Pro_Proveedores con Invitacion Directa',
 'Pro_Visualizaciones del Procedimiento',
 'Pro_Proveedores que Manifestaron Interes',
 'Pro_Respuestas al Procedimiento',
 'Pro_Respuestas Externas',
 'Pro_Conteo de Respuestas a Ofertas',
 'Pro_Proveedores Unicos con Respuestas',
 'Pro_Numero de Lotes',
 'Pro_Estado del Procedimiento',
 'Pro_ID Estado del Procedimiento',
 'Pro_Adjudicad

In [156]:
def cargueSecop2Predicciones(cnxn, cursor, df_temp):
    print('Proceso iniciado')
    
    for index, row in df_temp.iterrows():
        cursor.execute("""INSERT INTO Prod_SabanaAnalitica_Con_Predicciones ([Pro_Entidad],
                         [Pro_Nit Entidad],
                         [Pro_Departamento Entidad],
                         [Pro_Ciudad Entidad],
                         [Pro_OrdenEntidad],
                         [Pro_Entidad Centralizada],
                         [Pro_ID del Proceso],
                         [Pro_Referencia del Proceso],
                         [Pro_PCI],
                         [Pro_ID del Portafolio],
                         [Pro_Nombre del Procedimiento],
                         [Pro_Fase],
                         [Pro_Fecha de Publicacion del Proceso],
                         [Pro_Fecha de Ultima Publicación],
                         [Pro_Precio Base],
                         [Pro_Modalidad de Contratacion],
                         [Pro_Justificación Modalidad de Contratación],
                         [Pro_Duracion],
                         [Pro_Unidad de Duracion],
                         [Pro_Ciudad de la Unidad de Contratación],
                         [Pro_Nombre de la Unidad de Contratación],
                         [Pro_Proveedores Invitados],
                         [Pro_Proveedores con Invitacion Directa],
                         [Pro_Visualizaciones del Procedimiento],
                         [Pro_Proveedores que Manifestaron Interes],
                         [Pro_Respuestas al Procedimiento],
                         [Pro_Respuestas Externas],
                         [Pro_Conteo de Respuestas a Ofertas],
                         [Pro_Proveedores Unicos con Respuestas],
                         [Pro_Numero de Lotes],
                         [Pro_Estado del Procedimiento],
                         [Pro_ID Estado del Procedimiento],
                         [Pro_Adjudicado],
                         [Pro_ID Adjudicacion],
                         [Pro_CodigoProveedor],
                         [Pro_Departamento Proveedor],
                         [Pro_Ciudad Proveedor],
                         [Pro_Valor Total Adjudicacion],
                         [Pro_Nombre del Adjudicador],
                         [Pro_Nombre del Proveedor Adjudicado],
                         [Pro_NIT del Proveedor Adjudicado],
                         [Pro_Codigo Principal de Categoria],
                         [Pro_Estado de Apertura del Proceso],
                         [Pro_Tipo de Contrato],
                         [Pro_Subtipo de Contrato],
                         [Pro_URLProceso],
                         [Pro_Codigo Entidad],
                         [Pro_Estado Resumen],
                         [Cat_Extraccion],
                         [FueMultado],
                         [Prob predicción],
                         [Semaforo],
                         [Descrip Semaforo]) values(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""", 
                        str(row['Pro_Entidad']),
                         str(row['Pro_Nit Entidad']),
                         str(row['Pro_Departamento Entidad']),
                         str(row['Pro_Ciudad Entidad']),
                         str(row['Pro_OrdenEntidad']),
                         str(row['Pro_Entidad Centralizada']),
                         str(row['Pro_ID del Proceso']),
                         str(row['Pro_Referencia del Proceso']),
                         str(row['Pro_PCI']),
                         str(row['Pro_ID del Portafolio']),
                         str(row['Pro_Nombre del Procedimiento']),
                         str(row['Pro_Fase']),
                         str(row['Pro_Fecha de Publicacion del Proceso']),
                         str(row['Pro_Fecha de Ultima Publicación']),
                         str(row['Pro_Precio Base']),
                         str(row['Pro_Modalidad de Contratacion']),
                         str(row['Pro_Justificación Modalidad de Contratación']),
                         str(row['Pro_Duracion']),
                         str(row['Pro_Unidad de Duracion']),
                         str(row['Pro_Ciudad de la Unidad de Contratación']),
                         str(row['Pro_Nombre de la Unidad de Contratación']),
                         str(row['Pro_Proveedores Invitados']),
                         str(row['Pro_Proveedores con Invitacion Directa']),
                         str(row['Pro_Visualizaciones del Procedimiento']),
                         str(row['Pro_Proveedores que Manifestaron Interes']),
                         str(row['Pro_Respuestas al Procedimiento']),
                         str(row['Pro_Respuestas Externas']),
                         str(row['Pro_Conteo de Respuestas a Ofertas']),
                         str(row['Pro_Proveedores Unicos con Respuestas']),
                         str(row['Pro_Numero de Lotes']),
                         str(row['Pro_Estado del Procedimiento']),
                         str(row['Pro_ID Estado del Procedimiento']),
                         str(row['Pro_Adjudicado']),
                         str(row['Pro_ID Adjudicacion']),
                         str(row['Pro_CodigoProveedor']),
                         str(row['Pro_Departamento Proveedor']),
                         str(row['Pro_Ciudad Proveedor']),
                         str(row['Pro_Valor Total Adjudicacion']),
                         str(row['Pro_Nombre del Adjudicador']),
                         str(row['Pro_Nombre del Proveedor Adjudicado']),
                         str(row['Pro_NIT del Proveedor Adjudicado']),
                         str(row['Pro_Codigo Principal de Categoria']),
                         str(row['Pro_Estado de Apertura del Proceso']),
                         str(row['Pro_Tipo de Contrato']),
                         str(row['Pro_Subtipo de Contrato']),
                         str(row['Pro_URLProceso']),
                         str(row['Pro_Codigo Entidad']),
                         str(row['Pro_Estado Resumen']),
                         str(row['Cat_Extraccion']),
                         str(row['FueMultado']),
                         str(row['Prob predicción']),
                         str(row['Semaforo']),
                         str(row['Descrip Semaforo']))
        cnxn.commit()
    
    print('Datos cargados')

In [157]:
# 1. Se inicia la conexion con la base de datos
cnxn, cursor = iniciarSesion()
print("Sesion iniciada")

Sesion iniciada


In [154]:
df_usar = df_test[df_test['Semaforo'] != 0]
df_usar.shape

(8, 53)

In [155]:
df_usar['Pro_Fecha de Publicacion del Proceso'].head(16)

7738    2023-03-27
8503    2023-04-04
14014   2023-04-13
19848   2023-04-18
20055   2023-04-05
33703   2023-02-24
33704   2023-02-24
37208   2023-04-09
Name: Pro_Fecha de Publicacion del Proceso, dtype: datetime64[ns]

In [158]:
# 4. Se cargan los datos completos a la base SQL
k = 0
total = 0

while total < df_usar.shape[0]:
    cnxn, cursor = iniciarSesion()
    df_temp = df_usar[k*1000:min((k+1)*1000, df_usar.shape[0])]
    print(k)
    cargueSecop2Predicciones(cnxn, cursor, df_temp)
    print("Carga completa")
    k += 1
    total += df_temp.shape[0]
    print(total)
    cerrarCursor(cursor)

# 5. Se cierra el cursor
# cerrarCursor(cursor)
print("Proceso finalizado")

0
Proceso iniciado
Datos cargados
Carga completa
8
Proceso finalizado


In [82]:
def borrarSecop2Completo():
    cnxn.execute("""TRUNCATE TABLE Prod_SabanaAnalitica_Con_Predicciones""")
    cnxn.commit()
    
# 3. Se limipia la tabla para actualizar con los nuevos datos
borrarSecop2Completo()